<a href="https://colab.research.google.com/github/ftrotter/SQLClass/blob/main/Fake_Sales_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
!pip install faker
!pip install pymysql
import sqlalchemy
import pandas as pd
import getpass
import faker
import random
from datetime import datetime
from faker import Faker
fake = Faker()

#this is how we setup the db
passwd = getpass.getpass('  enter the cardremote db password:')
hostname = getpass.getpass('  enter the hostname:')
engine = sqlalchemy.create_engine('mysql+pymysql://collab:' + passwd + '@' + hostname +'.set.care:3306/selling')


#here are the parameters of the script
the_year = 2021 # all of the simulated data will be for this year

sales_people_count = 2 #This will determine how many sales ppl are included in this data generation run

min_sales_calls_per_day = 2
max_sales_calls_per_day = 25

unlucky_sales_days = [1,5] # no one likes to talk on monday or friday

good_day_success_rate = 70 # on good days, what is the call success rate?
bad_day_success_date = 20 # on bad days, what is the call success rate?

# given a day of the week, randomly determine how many calls were made that day
def random_sales_this_day_of_week(day_of_week_num, min_sales_calls_per_day, max_sales_calls_per_day, excluded_day_list = []):
  if day_of_week_num in excluded_day_list:
    return(0)
  else: 
    return(random.randrange(min_sales_calls_per_day,max_sales_calls_per_day))



# Given a day of the week... randomly decide if a given call worked
def did_this_call_work(day_of_week, fail_day_list = [1,5], good_day_success_rate = 70, bad_day_success_rate = 10): # lower success rates on monday and friday by default

  threshold = good_day_success_rate
  if day_of_week in fail_day_list:
    threshold = bad_day_success_rate
  chance = random.randrange(0,100)
  if chance > threshold: 
    return(True)
  else:
    return(False)



#Loop over all of the sales people
while sales_people_count > 0:

  sales_person_name = fake.name() 
  print(sales_person_name)

  year_day = 365


  # Save the given sales person here... 
  save_salesperson_sql = f"""
INSERT INTO selling.salesperson 
  (`id`, `salesperson_name`, `created_at`, `updated_at`) 
VALUES 
  (NULL, '{sales_person_name}', current_timestamp(), current_timestamp());

"""

  lastresult = engine.execute(save_salesperson_sql)
  salesperson_id = lastresult.lastrowid

  # And retain the id of the record for later use.

  # Loop over all of the days in a year... 
  while year_day > 0: 
    # = datetime.date(the_year, 1, 1) + datetime.timedelta(year_day - 1)
    d = datetime.strptime(str(the_year) + "-" + str(year_day), "%Y-%j")




    day_of_the_week = d.weekday()

    year_day = year_day - 1

    sales_calls_today = random_sales_this_day_of_week(day_of_the_week, min_sales_calls_per_day, max_sales_calls_per_day)
 
    # loop over every individual sales call
    while sales_calls_today > 0:
      sales_calls_today = sales_calls_today - 1
      if(did_this_call_work(day_of_the_week)):
        is_call_worked = 1
      else:
        is_call_worked = 0

      hour = random.randrange(7,19)
      min = random.randrange(1,60)
      seconds = random.randrange(1,60)

      date_for_mysql = d.strftime('%Y-%m-%d') + f" {hour}:{min}:{seconds}"

      insert_call_sql = f"""

INSERT INTO selling.callevent 
  (`id`, `salesperson_id`, `is_call_success`, `call_timedate`)
VALUES 
  (NULL, '{salesperson_id}', '{is_call_worked}', '{date_for_mysql}.000000');

"""
      lastresult = engine.execute(insert_call_sql)
      #print(f"person: {sales_person_name} date: {date_for_mysql} is_call_worked: {is_call_worked}  ")


  sales_people_count = sales_people_count - 1


# Now lets reorder to the callevent so that it looks like it was created in order... 

sql = [] # empty list to start 



sql.append('DROP TABLE IF EXISTS selling.ordered_callevent')

this_sql = f"""
CREATE TABLE selling.ordered_callevent AS 
SELECT 
	  `salesperson_id`, 
    `is_call_success`, 
    `call_timedate` 
FROM `callevent`
ORDER BY call_timedate

"""
sql.append(this_sql)

sql.append('DROP TABLE selling.callevent')

sql.append('CREATE TABLE selling.callevent SELECT * FROM ordered_callevent ORDER BY call_timedate')

sql.append('ALTER TABLE selling.callevent ADD `id` INT(11) NOT NULL AUTO_INCREMENT FIRST, ADD PRIMARY KEY (`id`) ')

sql.append('DROP TABLE selling.ordered_callevent')

for this_sql in sql:
  print('Running: ' +  this_sql)
  engine.execute(this_sql)




  enter the cardremote db password:··········
  enter the hostname:··········
Daniel Henderson
Richard Davis
Running: DROP TABLE IF EXISTS selling.ordered_callevent
Running: 
CREATE TABLE selling.ordered_callevent AS 
SELECT 
	  `salesperson_id`, 
    `is_call_success`, 
    `call_timedate` 
FROM `callevent`
ORDER BY call_timedate


Running: DROP TABLE selling.callevent
Running: CREATE TABLE selling.callevent SELECT * FROM ordered_callevent ORDER BY call_timedate
Running: ALTER TABLE selling.callevent ADD `id` INT(11) NOT NULL AUTO_INCREMENT FIRST, ADD PRIMARY KEY (`id`) 
Running: DROP TABLE selling.ordered_callevent
